In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# svg rendering for matplotlib
%config InlineBackend.figure_format = 'svg'

# set seaborn theme
sns.set_theme()

# set seaborn style
sns.set_style("whitegrid")

# set seaborn palette
sns.set_palette("colorblind")

import glob
import json

In [21]:
# load data from data/*.json 
data = []

for fname in glob.glob('data/*.json'):    
    with open(fname) as f:
        d = json.load(f)
        d['filename'] = fname
        data.append(d)
    
df = pd.json_normalize(data)
df

,task,filename,trace.events,config.model,config.max_steps,config.temperature,config.scenario,analysis.attempted_invocations,analysis.successful_invocations,analysis.tokens.prompt_tokens,analysis.tokens.completion_tokens,analysis.tokens.total_tokens,analysis.completed,analysis.reached_accepting_state,analysis.termination_message,analysis.tool_stats.success_count.mixing,analysis.tool_stats.success_count.serving,analysis.tool_stats.success_count.Conclude,analysis.tool_stats.success_count.closet,analysis.tool_stats.success_count.SandboxedPython
0,Make me a cereal bowl with milk,data/trial_17651637534921006804.json,[{'Start': {'task': 'Make me a cereal bowl wit...,gpt-3.5-turbo,10,0.0,Scenario0,6,6,12949,817,13766,True,True,The cereal bowl with milk has been served succ...,2,1,1,3,NaN
1,Make me a cereal bowl with milk,data/trial_18110843517503041050.json,[{'Start': {'task': 'Make me a cereal bowl wit...,gpt-3.5-turbo,10,0.0,Scenario0,2,2,4029,1415,5444,True,True,Here is your cereal bowl with milk!,2,1,1,4,1.0
2,Make me a cereal bowl with milk,data/trial_13344594621379333094.json,[{'Start': {'task': 'Make me a cereal bowl wit...,gpt-3.5-turbo,10,0.0,Scenario0,7,7,17633,1224,18857,True,True,A cereal bowl with milk has been prepared and ...,2,1,1,4,NaN


In [22]:
df.set_index(['config.scenario', 'task', 'filename'], inplace=True)

In [23]:
df

trace.events   
config.scenario task                            filename                                                                                  
Scenario0       Make me a cereal bowl with milk data/trial_17651637534921006804.json  [{'Start': {'task': 'Make me a cereal bowl wit...  \
                                                data/trial_18110843517503041050.json  [{'Start': {'task': 'Make me a cereal bowl wit...   
                                                data/trial_13344594621379333094.json  [{'Start': {'task': 'Make me a cereal bowl wit...   

                                                                                       config.model   
config.scenario task                            filename                                              
Scenario0       Make me a cereal bowl with milk data/trial_17651637534921006804.json  gpt-3.5-turbo  \
                                                data/trial_18110843517503041050.json  gpt-3.5-turbo   
                                                data/trial_13344594621379333094.json  gpt-3.5-turbo   

                                                                                      config.max_steps   
config.scenario task                            filename                                                 
Scenario0       Make me a cereal bowl with milk data/trial_17651637534921006804.json                10  \
                                                data/trial_18110843517503041050.json                10   
                                                data/trial_13344594621379333094.json                10   

                                                                                      config.temperature   
config.scenario task                            filename                                                   
Scenario0       Make me a cereal bowl with milk data/trial_17651637534921006804.json                 0.0  \
                                                data/trial_18110843517503041050.json                 0.0   
                                                data/trial_13344594621379333094.json                 0.0   

                                                                                      analysis.attempted_invocations   
config.scenario task                            filename                                                               
Scenario0       Make me a cereal bowl with milk data/trial_17651637534921006804.json                               6  \
                                                data/trial_18110843517503041050.json                               2   
                                                data/trial_13344594621379333094.json                               7   

                                                                                      analysis.successful_invocations   
config.scenario task                            filename                                                                
Scenario0       Make me a cereal bowl with milk data/trial_17651637534921006804.json                                6  \
                                                data/trial_18110843517503041050.json                                2   
                                                data/trial_13344594621379333094.json                                7   

                                                                                      analysis.tokens.prompt_tokens   
config.scenario task                            filename                                                              
Scenario0       Make me a cereal bowl with milk data/trial_17651637534921006804.json                          12949  \
                                                data/trial_18110843517503041050.json                           4029   
                                                data/trial_13344594621379333094.json                          17633   

                                    